In [2]:
#Reference: https://www.kaggle.com/danrusei/10-monkey-keras-transfer-learning-resnet50

import numpy as np 
import pandas as pd 

from tensorflow.contrib.keras.api.keras.callbacks import Callback
from tensorflow.contrib.keras.api.keras.preprocessing.image import ImageDataGenerator
from tensorflow.contrib.keras import backend
from keras.optimizers import Adam
from glob import glob
import os

Using TensorFlow backend.


In [3]:
training_dir = "C:/Users/avadh/Desktop/CMPE 256/Dataset/training_set"
test_dir = "C:/Users/avadh/Desktop/CMPE 256/Dataset/validation_set"

In [4]:
# useful for getting number of files
train_count = glob(training_dir + '/*/*.jp*g')
test_count = glob(test_dir + '/*/*.jp*g')

In [5]:
# getting the number of classes i.e. type of fruits
folders = glob(training_dir + '/*')
num_classes = len(folders)
print(num_classes)

5


In [14]:
from keras.applications import ResNet50
from keras.models import Sequential
from keras.layers import Dense, Flatten, GlobalAveragePooling2D
import keras
my_new_model = Sequential()

my_new_model.add(ResNet50(include_top=False, pooling='avg', weights='imagenet'))
my_new_model.add(Dense(4096,activation='relu'))
my_new_model.add(keras.layers.Dropout(0.5))

my_new_model.add(Dense(num_classes, activation='softmax'))

# my_new_model.layers[0].trainable = False
for layers in my_new_model.layers[-2:]:
    layers.trainable=True


In [15]:
my_new_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resnet50 (Model)             (None, 2048)              23587712  
_________________________________________________________________
dense_5 (Dense)              (None, 4096)              8392704   
_________________________________________________________________
dropout_2 (Dropout)          (None, 4096)              0         
_________________________________________________________________
dense_6 (Dense)              (None, 5)                 20485     
Total params: 32,000,901
Trainable params: 31,947,781
Non-trainable params: 53,120
_________________________________________________________________


In [16]:
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.vgg16 import preprocess_input
input_size = (64,64)
batch_size = 32

training_datagen = ImageDataGenerator(
                                    rescale=1./255,   # all pixel values will be between 0 an 1
                                    shear_range=0.2, 
                                    zoom_range=0.2,
                                    horizontal_flip = True,
                                    preprocessing_function=preprocess_input)

test_datagen = ImageDataGenerator(
                                    rescale=1./255,   # all pixel values will be between 0 an 1
                                    preprocessing_function=preprocess_input)


training_generator = training_datagen.flow_from_directory(training_dir, target_size = input_size, batch_size = batch_size, class_mode = 'categorical')
test_generator = test_datagen.flow_from_directory(test_dir, target_size = input_size, batch_size = batch_size, class_mode= 'categorical')


Found 7721 images belonging to 5 classes.
Found 856 images belonging to 5 classes.


In [17]:
import keras
opt = keras.optimizers.Adam(lr=1e-3, decay=1e-6) 
my_new_model.compile(optimizer= opt, loss='categorical_crossentropy', metrics=['accuracy'])


In [18]:
test_generator.class_indices

{'drawings': 0,
 'engraving': 1,
 'iconography': 2,
 'painting': 3,
 'sculpture': 4}

In [19]:
model_info = my_new_model.fit_generator(training_generator,
                         steps_per_epoch=7721/batch_size,
                         epochs=2,
                         validation_data=test_generator,
                         validation_steps=856/batch_size,
                         workers=12)

Epoch 1/2
 55/241 [=====>........................] - ETA: 32:06 - loss: 1.2838 - acc: 0.5506

C:\Users\avadh\Anaconda3\envs\tensorflow\lib\site-packages\PIL\Image.py:931: UserWarning: Palette images with Transparency   expressed in bytes should be converted to RGBA images
  'to RGBA images')


179/241 [=====================>........] - ETA: 9:49 - loss: 0.8721 - acc: 0.6988

C:\Users\avadh\Anaconda3\envs\tensorflow\lib\site-packages\PIL\TiffImagePlugin.py:747: UserWarning: Possibly corrupt EXIF data.  Expecting to read 6029312 bytes but only got 0. Skipping tag 0
  " Skipping tag %s" % (size, len(data), tag))
C:\Users\avadh\Anaconda3\envs\tensorflow\lib\site-packages\PIL\TiffImagePlugin.py:747: UserWarning: Possibly corrupt EXIF data.  Expecting to read 1311848 bytes but only got 785. Skipping tag 0
  " Skipping tag %s" % (size, len(data), tag))
C:\Users\avadh\Anaconda3\envs\tensorflow\lib\site-packages\PIL\TiffImagePlugin.py:764: UserWarning: Corrupt EXIF data.  Expecting to read 12 bytes but only got 8. 
  warnings.warn(str(msg))


242/241 [==============================] - 2171s 9s/step - loss: 0.8021 - acc: 0.7238 - val_loss: 1.0351 - val_acc: 0.6928
Epoch 2/2
242/241 [==============================] - 1649s 7s/step - loss: 0.5998 - acc: 0.8007 - val_loss: 1.5206 - val_acc: 0.7290
